# Denoising 모델 학습 노트북

**사전 준비:**
1. `colab_generate_datasets.ipynb`를 실행하여 `test_1_noise` 등 학습 데이터셋 생성을 완료해주세요.
2. `colab_prepare_dataset.ipynb`를 실행하여 `dataset.zip` 파일과 파일 ID를 생성해주세요.



In [ ]:
# 1. Google Drive 마운트 및 경로 설정
from google.colab import drive
drive.mount('/content/drive')

# 2. [수정 필요] 프로젝트 경로 설정
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/week5" # 예시 경로
import os
os.chdir(PROJECT_PATH)

# 3. 필요 라이브러리 설치
%pip install loguru tqdm gdown



In [ ]:
# 4. (자동화) gdown으로 데이터셋 다운로드 및 압축 해제 (최고 속도)
import gdown
import zipfile
from pathlib import Path

# --- 설정 ---
# [수정 필요] `colab_prepare_dataset.ipynb` 실행 후 얻은 파일 ID를 여기에 붙여넣어주세요.
DATASET_FILE_ID = "YOUR_DATASET_FILE_ID_HERE" 

LOCAL_ZIP_PATH = Path("/content/dataset.zip")
LOCAL_DATASET_PATH = Path("/content/dataset")

# --- 다운로드 및 압축 해제 ---
print("gdown을 사용하여 데이터셋 다운로드를 시작합니다...")
gdown.download(id=DATASET_FILE_ID, output=str(LOCAL_ZIP_PATH), quiet=False)
print("다운로드 완료.")

print(f"'{LOCAL_ZIP_PATH}' 파일의 압축을 '{LOCAL_DATASET_PATH}'에 해제합니다...")
with zipfile.ZipFile(str(LOCAL_ZIP_PATH), 'r') as zip_ref:
    zip_ref.extractall(path="/content/")
print("압축 해제 완료.")

# --- 환경 변수 설정 ---
os.environ['DATA_ROOT'] = str(LOCAL_DATASET_PATH)



In [ ]:
# 5. Denoising 모델 학습 실행
!python code_denoising/train.py --model_type dncnn --gpu 0 --train_dataset_name "test_1_noise" --run_dir_name "logs_denoising"


# Denoising 모델 학습 노트북

**사전 준비:** `colab_generate_datasets.ipynb`를 실행하여 학습 데이터셋 생성을 먼저 완료해주세요.



In [ ]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. [수정 필요] 프로젝트 경로 설정
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/week5" # 예시 경로
import os
os.chdir(PROJECT_PATH)

# 3. 필요 라이브러리 설치
%pip install loguru tqdm



In [ ]:
# 4. (자동화) 필요한 데이터셋만 Colab 로컬 런타임으로 복사 (파일 단위 진행률 표시)
import os
import shutil
from tqdm.notebook import tqdm
from pathlib import Path

# --- 설정 ---
# 2번 셀에서 정의한 PROJECT_PATH 변수를 사용합니다.
DRIVE_DATASET_PATH = Path(PROJECT_PATH) / 'dataset'
LOCAL_DATASET_PATH = Path('/content/dataset')
REQUIRED_DIRS = ['train', 'val', 'test_1_noise']

# --- 디렉토리 준비 ---
print(f"'{LOCAL_DATASET_PATH}'으로 필요한 데이터셋 복사를 준비합니다...")
if LOCAL_DATASET_PATH.exists():
    shutil.rmtree(LOCAL_DATASET_PATH)
    print(f"기존 '{LOCAL_DATASET_PATH}' 폴더를 삭제했습니다.")
LOCAL_DATASET_PATH.mkdir(parents=True)

# --- 1단계: 전체 파일 개수 세기 ---
print("전체 파일 목록을 스캔 중입니다... (Google Drive에서 이 작업은 몇 분 정도 소요될 수 있습니다)")
total_files = 0
for dir_name in REQUIRED_DIRS:
    src_dir = DRIVE_DATASET_PATH / dir_name
    if src_dir.exists():
        total_files += sum(len(files) for _, _, files in os.walk(src_dir))
print(f"총 {total_files}개의 파일을 복사합니다.")

# --- 2단계: 파일 단위로 복사하며 진행률 표시 ---
with tqdm(total=total_files, desc="파일 복사 진행률") as pbar:
    for dir_name in REQUIRED_DIRS:
        src_dir = DRIVE_DATASET_PATH / dir_name
        if not src_dir.exists():
            print(f"경고: 소스 폴더를 찾을 수 없어 건너뜁니다 - {src_dir}")
            continue

        for root, dirs, files in os.walk(src_dir):
            relative_path = Path(root).relative_to(DRIVE_DATASET_PATH)
            local_dir = LOCAL_DATASET_PATH / relative_path
            local_dir.mkdir(parents=True, exist_ok=True)
            
            for filename in files:
                src_path = Path(root) / filename
                dst_path = local_dir / filename
                shutil.copy2(src_path, dst_path)
                pbar.update(1)

print("데이터셋 복사 완료!")

# --- 환경 변수 설정 ---
os.environ['DATA_ROOT'] = str(LOCAL_DATASET_PATH)



In [ ]:
# 5. Denoising 모델 학습 실행
!python code_denoising/train.py --model_type dncnn --gpu 0 --train_dataset_name "test_1_noise" --run_dir_name "logs_denoising"


# Denoising 모델 학습 노트북

**사전 준비:** `colab_generate_datasets.ipynb`를 실행하여 학습 데이터셋 생성을 먼저 완료해주세요.



In [ ]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. [수정 필요] 프로젝트 경로 설정
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/week5" # 예시 경로
import os
os.chdir(PROJECT_PATH)

# 3. 필요 라이브러리 설치
%pip install loguru tqdm



In [ ]:
# 4. (자동화) 필요한 데이터셋만 Colab 로컬 런타임으로 복사 (최적화 버전)
import os
import shutil
from tqdm.notebook import tqdm
from pathlib import Path

# --- 설정 ---
# 2번 셀에서 정의한 PROJECT_PATH 변수를 사용합니다.
DRIVE_DATASET_PATH = Path(PROJECT_PATH) / 'dataset'
LOCAL_DATASET_PATH = Path('/content/dataset')

# 이 노트북에 필요한 특정 폴더 목록
REQUIRED_DIRS = ['train', 'val', 'test_1_noise']

# --- 디렉토리 준비 ---
print(f"'{LOCAL_DATASET_PATH}'으로 필요한 데이터셋 복사를 준비합니다...")
if LOCAL_DATASET_PATH.exists():
    shutil.rmtree(LOCAL_DATASET_PATH)
    print(f"기존 '{LOCAL_DATASET_PATH}' 폴더를 삭제했습니다.")
LOCAL_DATASET_PATH.mkdir(parents=True)

# --- 필요한 폴더들만 복사 ---
print(f"다음 폴더들을 복사합니다: {REQUIRED_DIRS}")
for dir_name in tqdm(REQUIRED_DIRS, desc="폴더 복사 진행률"):
    src_path = DRIVE_DATASET_PATH / dir_name
    dst_path = LOCAL_DATASET_PATH / dir_name
    
    if src_path.exists():
        shutil.copytree(src_path, dst_path)
    else:
        print(f"경고: 소스 폴더를 찾을 수 없습니다 - {src_path}")

print("데이터셋 복사 완료!")

# --- 환경 변수 설정 ---
os.environ['DATA_ROOT'] = str(LOCAL_DATASET_PATH)



In [ ]:
# 5. Denoising 모델 학습 실행
!python code_denoising/train.py --model_type dncnn --gpu 0 --train_dataset_name "test_1_noise" --run_dir_name "logs_denoising"


# Denoising 모델 학습 노트북

**사전 준비:** `colab_generate_datasets.ipynb`를 실행하여 학습 데이터셋 생성을 먼저 완료해주세요.



In [ ]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. [수정 필요] 프로젝트 경로 설정
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/week5" # 예시 경로
import os
os.chdir(PROJECT_PATH)

# 3. 필요 라이브러리 설치
%pip install loguru tqdm



In [ ]:
# 4. (자동화) 데이터셋을 Colab 로컬 런타임으로 복사 (개선된 방식)
import os
import shutil
from tqdm.notebook import tqdm
from pathlib import Path

# --- 설정 ---
# 2번 셀에서 정의한 PROJECT_PATH 변수를 사용합니다.
DRIVE_DATASET_PATH = Path(PROJECT_PATH) / 'dataset'
LOCAL_DATASET_PATH = Path('/content/dataset')

# --- 디렉토리 준비 ---
print(f"'{DRIVE_DATASET_PATH}'에서 '{LOCAL_DATASET_PATH}'으로 데이터셋 복사를 준비합니다...")
if LOCAL_DATASET_PATH.exists():
    shutil.rmtree(LOCAL_DATASET_PATH)
    print(f"기존 '{LOCAL_DATASET_PATH}' 폴더를 삭제했습니다.")
LOCAL_DATASET_PATH.mkdir(parents=True)

# --- 1단계: 전체 파일 개수 세기 (rglob 병목 현상 해결) ---
print("전체 파일 목록을 스캔 중입니다... (Google Drive에서 이 작업은 몇 분 정도 소요될 수 있습니다)")
total_files = sum(len(files) for _, _, files in os.walk(DRIVE_DATASET_PATH))
print(f"총 {total_files}개의 파일을 복사합니다.")

# --- 2단계: tqdm 프로그레스 바와 함께 파일 복사 ---
with tqdm(total=total_files, desc="파일 복사 진행률") as pbar:
    for root, dirs, files in os.walk(DRIVE_DATASET_PATH):
        # 대상 디렉토리 경로 생성
        relative_path = Path(root).relative_to(DRIVE_DATASET_PATH)
        local_dir = LOCAL_DATASET_PATH / relative_path
        local_dir.mkdir(parents=True, exist_ok=True)
        
        # 현재 디렉토리의 파일들을 복사
        for filename in files:
            src_path = Path(root) / filename
            dst_path = local_dir / filename
            shutil.copy2(src_path, dst_path)
            pbar.update(1)

print("데이터셋 복사 완료!")

# --- 환경 변수 설정 ---
os.environ['DATA_ROOT'] = str(LOCAL_DATASET_PATH)



In [ ]:
# 5. Denoising 모델 학습 실행
!python code_denoising/train.py --model_type dncnn --gpu 0 --train_dataset_name "test_1_noise" --run_dir_name "logs_denoising"


# Denoising 모델 학습 노트북

**사전 준비:** `colab_generate_datasets.ipynb`를 실행하여 학습 데이터셋 생성을 먼저 완료해주세요.



In [ ]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. [수정 필요] 프로젝트 경로 설정
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/week5" # 예시 경로
import os
os.chdir(PROJECT_PATH)

# 3. 필요 라이브러리 설치
%pip install loguru



In [ ]:
# 4. (자동화) 데이터셋을 Colab 로컬 런타임으로 복사 (속도 향상)
import os
import shutil
from tqdm.notebook import tqdm
from pathlib import Path

# 2번 셀에서 정의한 PROJECT_PATH 변수를 사용합니다.
DRIVE_DATASET_PATH = Path(PROJECT_PATH) / 'dataset'
LOCAL_DATASET_PATH = Path('/content/dataset')

print(f"'{DRIVE_DATASET_PATH}'에서 '{LOCAL_DATASET_PATH}'으로 데이터셋 복사를 시작합니다...")

# 이미 폴더가 있다면 삭제
if LOCAL_DATASET_PATH.exists():
    shutil.rmtree(LOCAL_DATASET_PATH)
    print(f"기존 '{LOCAL_DATASET_PATH}' 폴더를 삭제했습니다.")

# 복사할 모든 파일 목록 가져오기
files_to_copy = list(DRIVE_DATASET_PATH.rglob('*'))
print(f"총 {len(files_to_copy)}개의 파일을 복사합니다.")

# tqdm을 사용하여 파일 복사 진행 상황 표시
for src_path in tqdm(files_to_copy, desc="Copying files"):
    if src_path.is_file():
        # 대상 경로 계산
        relative_path = src_path.relative_to(DRIVE_DATASET_PATH)
        dst_path = LOCAL_DATASET_PATH / relative_path
        
        # 대상 디렉토리 생성
        dst_path.parent.mkdir(parents=True, exist_ok=True)
        
        # 파일 복사
        shutil.copy2(src_path, dst_path)

print("데이터셋 복사 완료!")

# 학습 스크립트가 로컬 데이터셋을 보도록 환경 변수 설정
os.environ['DATA_ROOT'] = str(LOCAL_DATASET_PATH)



In [ ]:
# 5. Denoising 모델 학습 실행
!python code_denoising/train.py --model_type dncnn --gpu 0 --train_dataset_name "test_1_noise" --run_dir_name "logs_denoising"


# Denoising 모델 학습 노트북

**사전 준비:** `colab_generate_datasets.ipynb`를 실행하여 학습 데이터셋 생성을 먼저 완료해주세요.

